<a href="https://colab.research.google.com/github/RosalesLuis123/IA/blob/main/Lab5/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [11]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class NNEMNIST(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NNEMNIST, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 300)
        # self.fc2 = nn.Linear(800, 150)
        self.fc2 = nn.Linear(300, num_classes)

        # self.fc1 = nn.Linear(input_size, 47)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = torch.relu(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        # x = torch.sigmoid(x)
        # x = self.fc3(x)
        return x

In [57]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 63
num_classes = 2
learning_rate = 0.001
batch_size = 500000
num_epochs = 5

cpu


In [37]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [59]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Especifica la ruta al archivo CSV en tu Google Drive
csv_file_path = '/content/gdrive/MyDrive/Laboratorio5/lol.csv'

# Carga el conjunto de datos CSV
data = pd.read_csv(csv_file_path)

X = data.iloc[:, 1:-1]  # Todas las filas, desde la segunda columna hasta la penúltima
y = data.iloc[:, -1]  # Todas las filas, solo la última columna

# Convierte los datos a tensores de PyTorch
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# Crea un conjunto de datos TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Divide el conjunto de datos en conjunto de entrenamiento y conjunto de prueba
# Aquí puedes ajustar la proporción de datos para entrenamiento y prueba según sea necesario
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define el tamaño del lote y crea los DataLoader
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Imprime la longitud de los conjuntos de datos
print("Número de muestras en el conjunto de entrenamiento:", len(train_dataset))
print("Número de muestras en el conjunto de prueba:", len(test_dataset))


Número de muestras en el conjunto de entrenamiento: 367525
Número de muestras en el conjunto de prueba: 91882


In [47]:
X

,Time,Player1_X,Player1_Y,Player2_X,Player2_Y,Player3_X,Player3_Y,Player4_X,Player4_Y,Player5_X,...,Distance_6_7,Distance_6_8,Distance_6_9,Distance_6_10,Distance_7_8,Distance_7_9,Distance_7_10,Distance_8_9,Distance_8_10,Distance_9_10
0,0,554.000000,581.000000,593.000000,464.000000,557.000000,345.000000,458.000000,271.000000,335.000000,...,123.328829,236.019067,324.524267,381.188930,124.326184,235.529191,323.402226,123.600162,234.648674,123.016259
1,1,596.566667,751.083333,753.550000,576.266667,663.883333,455.516667,594.266667,340.750000,477.616667,...,206.377917,317.620765,398.464944,352.764776,114.265467,208.005594,212.190015,102.462882,161.464004,120.356600
2,2,639.133333,921.166667,914.100000,688.533333,770.766667,566.033333,730.533333,410.500000,620.233333,...,292.009208,425.035137,538.710973,422.569696,133.519741,247.509813,144.642352,120.403636,88.876494,130.992536
3,3,681.700000,1091.250000,1074.650000,800.800000,877.650000,676.550000,866.800000,480.250000,762.850000,...,378.474041,543.158128,706.839163,554.681424,172.539191,330.844892,179.664750,165.142908,23.349946,152.161099
4,4,724.266667,1261.333333,1235.200000,913.066667,984.533333,787.066667,1003.066667,550.000000,905.466667,...,465.307974,666.318696,887.136169,715.375420,221.095796,433.425923,281.244804,220.961706,63.318174,180.187618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459402,1016,2512.866667,11896.000000,2050.333333,11544.000000,10844.800000,1633.200000,6612.600000,7280.533333,6612.600000,...,6405.249809,11900.077544,7679.229450,12179.584381,5505.489785,1566.879793,7289.087073,4520.089254,5612.027286,7562.516578
459403,1017,2514.900000,11893.750000,2029.000000,11545.000000,10834.350000,1627.900000,6601.450000,7264.900000,6601.450000,...,6407.792722,11903.895166,7683.561913,12180.284396,5506.981257,1567.367557,7290.266515,4520.162262,5612.929153,7562.516578
459404,1018,2516.933333,11891.500000,2007.666667,11546.000000,10823.900000,1622.600000,6590.300000,7249.266667,6590.300000,...,6410.340893,11907.715988,7687.897193,12180.987691,5508.472749,1567.856266,7291.446002,4520.235455,5613.831024,7562.516578
459405,1019,2518.966667,11889.250000,1986.333333,11547.000000,10813.450000,1617.300000,6579.150000,7233.633333,6579.150000,...,6412.894316,11911.540007,7692.235286,12181.694264,5509.964262,1568.345918,7292.625534,4520.308833,5614.732900,7562.516578


In [68]:
for c in train_dataset:
  print(c)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
         4820.0034,  2728.3887,  1124.6377,  2575.6985,  2481.3469,  8021.2529,
         6133.9873,  1896.2604,  2615.5752,  4321.7124,  5508.2886,  6023.8135,
         8261.0234,  6225.4956,  4560.3638,  2812.9060,  4501.4468,  4756.1143,
         8932.4014,  6845.3145,  3475.0291,  1752.5656,  1943.4259,  6960.2896,
         5032.4438,   841.5113,   695.3660,  5583.0547,  3896.4971,  1035.3456,
         6038.2114,  4475.2280,  1463.4899,  2097.0298,  6133.6592,  4240.7344]), tensor(0))
(tensor([ 1021.0000,  6200.2998,  9102.0166,  4635.7002,  9144.7334,  1712.1333,
        10329.4834,  5126.4165,  4639.2334,  6935.0332,  8522.3838,  2780.5500,
        11291.6504,  9644.9004,  5626.0000,  8154.5332,  8194.0996, 13384.3662,
         4845.6167,  7952.2168,  7883.9331,  1565.1830,  4652.9897,  4590.1699,
          935.8461,  4060.6877,  4893.6655,  2154.8413,  8350.3145,  2133.7617,
         3154.5007,  4532.1445,  2382.0

In [64]:
input_size = 66  # Ajusta el tamaño de entrada para que coincida con el número de características en tus datos
num_classes = 2  # Ajusta el número de clases según tu problema
model = NNEMNIST(input_size=input_size, num_classes=num_classes).to(device)


In [65]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [66]:
# Train Network
for epoch in range(num_epochs):
    train_loss, train_acc = [], []

    # bar = tqdm(dataloader['train'])
    bar = tqdm(train_loader)
    for batch_idx, (data, targets) in enumerate(bar):
    # for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # data, targets = batch.data, batch.targets
        # data, targets = data.to(device), targets.to(device)

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        # print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        print(scores.shape)
        print(targets.shape)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        train_loss.append(loss.item())
        acc = (targets == torch.argmax(scores, axis=1)).sum().item() / len(targets)
        train_acc.append(acc)
        bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")


loss 1722.33521 acc 0.07812:   0%|          | 1/5743 [00:00<28:50,  3.32it/s]

torch.Size([64, 2])
torch.Size([64])


loss 309.86516 acc 0.90489:   0%|          | 23/5743 [00:00<01:35, 60.17it/s]

torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])


loss 186.88703 acc 0.90853:   1%|          | 37/5743 [00:00<01:08, 82.91it/s]

...

torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])


loss 0.15441 acc 0.94135: 100%|█████████▉| 5734/5743 [01:01<00:00, 91.04it/s]

torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])


loss 0.15441 acc 0.94134: 100%|██████████| 5743/5743 [01:01<00:00, 93.06it/s]

torch.Size([37, 2])
torch.Size([37])


In [67]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 94.13
Accuracy on test set: 94.02
